In [12]:
#mette insieme i modelli migliori per ogni funzione
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import random as rd
from sklearn.metrics import matthews_corrcoef, f1_score, confusion_matrix, accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
from pickle import dump, load
import ast

In [13]:
kos = pd.read_csv('../Matrices/genome_ko_all.csv').set_index('Genome')
functions = pd.read_csv('../Matrices/function_db.csv').set_index('genome_id')
functions.insert(0, 'ID', functions['Unnamed: 0'])
functions.drop('Unnamed: 0', axis = 1, inplace = True)

In [14]:
data = kos.merge(functions,left_index=True, right_index = True)

In [15]:
ko_input = data[kos.columns].drop('Species', axis = 1)
function_input = data[functions.columns].drop(['scientific_name', 'ID'], axis = 1)
y_col = [i for i in function_input.columns if function_input[i].sum() > 3]
y_input = function_input[y_col]

In [18]:
chosen_models = pd.read_csv('../final_tool.csv').set_index('functional_class')

In [19]:
chosen_models

,model,score,parameters
functional_class,,,
photosynthetic_cyanobacteria,svm,1.000000,"SVC(C=1, cache_size=1000, kernel='linear', ran..."
methanogenesis_by_CO2_reduction_with_H2,svm,1.000000,"SVC(C=1, cache_size=1000, kernel='linear', ran..."
methanogenesis_by_reduction_of_methyl_compounds_with_H2,svm,1.000000,"SVC(C=1, cache_size=1000, kernel='linear', ran..."
arsenite_oxidation_detoxification,svm,1.000000,"SVC(C=1, cache_size=1000, kernel='linear', ran..."
mammal_gut,svm,0.971230,"SVC(C=10, cache_size=1000, random_state=1)"
...,...,...,...
dissimilatory_arsenate_reduction,lr,0.774056,"LogisticRegression(C=0.01, max_iter=10000, ran..."
aerobic_anoxygenic_phototrophy,lr,0.763690,"LogisticRegression(C=1, max_iter=10000, random..."
chitinolysis,lr,0.748022,"LogisticRegression(C=10, max_iter=10000, penal..."


In [20]:
chosen_models.loc['mammal_gut']

model                                                svm
score                                            0.97123
parameters    SVC(C=10, cache_size=1000, random_state=1)
Name: mammal_gut, dtype: object

In [22]:
discarded_class = [i for i in function_input.columns if i not in y_input.columns]
discarded_class

['anammox', 'chloroplasts', 'nonphotosynthetic_cyanobacteria']

In [23]:
def classifier(function, x,y, chosen_pars):
    x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size = 0.2, random_state=1)
    sc = StandardScaler()
    x_train_normalized = sc.fit_transform(x_train)
    print(x_train_normalized.shape)
    best_model = eval(chosen_pars)
    print(best_model)
    best_model_fitted = best_model.fit(x_train_normalized, y_train)
    # save the model
    dump(best_model_fitted, open('model_'+function+'.sav', 'wb'))
    # save the scaler
    dump(sc, open('scaler_'+function+'.sav', 'wb'))
    print(function + ': classifier saved!')
    return  

In [29]:
#train models with the best parameters and save them(and scalers); NN models are optimized elsewhere
for func in list(y_input.columns):
    if chosen_models.loc[func]['model'] !="nn":
        print(func)
        classifier(func, ko_input, y_input[func], chosen_models.loc[func]['parameters'])

acetoclastic_methanogenesis
(11490, 11469)
LogisticRegression(C=0.001, max_iter=10000, random_state=1, solver='liblinear')
acetoclastic_methanogenesis: classifier saved!
methanogenesis_by_disproportionation_of_methyl_groups
(11490, 11469)
RandomForestClassifier(n_jobs=4, random_state=1)
methanogenesis_by_disproportionation_of_methyl_groups: classifier saved!
methanogenesis_using_formate
(11490, 11469)
SVC(C=1, cache_size=1000, kernel='linear', random_state=1)
methanogenesis_using_formate: classifier saved!
methanogenesis_by_CO2_reduction_with_H2
(11490, 11469)
SVC(C=1, cache_size=1000, kernel='linear', random_state=1)


KeyboardInterrupt: 

test on validation data

In [11]:
classes = ['anoxygenic_photoautotrophy_Fe_oxidizing', 'oil_bioremediation', 'dark_sulfite_oxidation', 'arsenate_respiration', 'manganese_respiration', 'dark_sulfur_oxidation', 'knallgas_bacteria', 'reductive_acetogenesis', 'dark_iron_oxidation', 'dark_thiosulfate_oxidation', 'chlorate_reducers', 'iron_respiration', 'anoxygenic_photoautotrophy_H2_oxidizing', 'nitrate_denitrification', 'chitinolysis', 'aerobic_anoxygenic_phototrophy', 'denitrification', 'dissimilatory_arsenate_reduction', 'dark_sulfide_oxidation', 'ureolysis', 'cellulolysis', 'thiosulfate_respiration', 'nitrous_oxide_denitrification', 'plastic_degradation', 'sulfur_respiration', 'aromatic_hydrocarbon_degradation', 'acetoclastic_methanogenesis', 'xylanolysis', 'sulfite_respiration', 'fumarate_respiration', 'dark_hydrogen_oxidation', 'nitrification', 'methanol_oxidation', 'sulfate_respiration', 'dark_oxidation_of_sulfur_compounds', 'nitrite_denitrification', 'arsenate_detoxification', 'anoxygenic_photoautotrophy_S_oxidizing', 'nitrate_respiration', 'nitrite_respiration', 'aromatic_compound_degradation', 'nitrate_ammonification', 'ligninolysis', 'nitrite_ammonification', 'phototrophy', 'respiration_of_sulfur_compounds', 'anoxygenic_photoautotrophy', 'methylotrophy', 'nitrogen_fixation', 'invertebrate_parasites', 'nitrogen_respiration', 'photoheterotrophy', 'chemoheterotrophy', 'nitrate_reduction', 'aerobic_ammonia_oxidation', 'predatory_or_exoparasitic', 'methanogenesis_using_formate', 'plant_pathogen', 'human_pathogens_meningitis', 'human_pathogens_gastroenteritis', 'hydrocarbon_degradation', 'manganese_oxidation', 'animal_parasites_or_symbionts', 'human_pathogens_all', 'photoautotrophy', 'human_pathogens_septicemia', 'aerobic_chemoheterotrophy', 'human_associated', 'aliphatic_non_methane_hydrocarbon_degradation', 'human_pathogens_pneumonia', 'fermentation', 'human_pathogens_diarrhea', 'mammal_gut', 'methanotrophy', 'human_gut', 'intracellular_parasites', 'methanogenesis_by_CO2_reduction_with_H2', 'methanogenesis_by_disproportionation_of_methyl_groups', 'methanogenesis_by_reduction_of_methyl_compounds_with_H2', 'hydrogenotrophic_methanogenesis', 'oxygenic_photoautotrophy', 'aerobic_nitrite_oxidation', 'methanogenesis', 'arsenite_oxidation_detoxification', 'arsenite_oxidation_energy_yielding', 'fish_parasites', 'dissimilatory_arsenite_oxidation', 'photosynthetic_cyanobacteria', 'human_pathogens_nosocomia']

In [16]:
#load validation data
to_validate = pd.read_csv('./input_classifier/ko_df_validation.csv').set_index('Unnamed: 0')

In [17]:
def get_validation_set(to_validate, training_set):
    training_kos = training_set.columns
    validation_kos = to_validate.columns
    if len(set(training_kos).intersection(set(validation_kos))) == 0:
        print('no common kos between provided ones and training')
        return
    else:
        common = list(set(training_kos).intersection(set(validation_kos)))
        print('{} common kos'.format(len(common)))
        common_table = to_validate[common]
        #remove orthologs in validation not in the training
        to_remove = set(validation_kos) - set(training_kos)
        print('{} kos present in user set but not in training set will be removed'.format(len(to_remove)))
        missing = list(set(training_kos) - set(validation_kos))
        print('{} kos missing from the users et will be add to train the classifiers'.format(len(missing)))
        #missing_df = pd
        missed = pd.DataFrame(0, index = to_validate.index, columns= missing)
        to_submit = common_table.merge(missed, left_index = True, right_index = True)
        #print(list(validation_kos))
        to_submit = to_submit[list(training_kos)] #change order columns
        print('Shape of training dataset: {}, Shape of user dataset: {}'.format(training_set.shape, to_submit.shape))
        if list(to_submit.columns) != list(training_set.columns): 
            print('ERROR')
            return
    return to_submit


In [23]:
validation = get_validation_set(to_validate, ko_input)
data_for_validation = validation.merge(functions,left_index=True, right_index = True)
ko_validation = data_for_validation[validation.columns]
function_validation = data_for_validation[classes]

In [62]:
results_per_class = {}
scores = {}
for c in classes[::-1]:
    print(c)
    model = load(open('model_'+c+'.sav', 'rb'))
    scaler = load(open('scaler_'+c+'.sav', 'rb'))
    to_validate_norm = scaler.transform(ko_validation)
    pred = model.predict(to_validate_norm)


    results_per_class[c] = pred
    scores[c] = [matthews_corrcoef(function_validation[c], pred), f1_score(function_validation[c], pred, zero_division=0), confusion_matrix(function_validation[c], pred), 
                 accuracy_score(function_validation[c], pred)]
final_scores = pd.DataFrame(scores).T
final_scores.to_csv("Validation_scores.csv")

human_pathogens_nosocomia
photosynthetic_cyanobacteria
dissimilatory_arsenite_oxidation
fish_parasites
arsenite_oxidation_energy_yielding
arsenite_oxidation_detoxification
